In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
# read all the words
words = open("names.txt", "r").read().splitlines()
print("total words : ", len(words))


total words :  32033


In [14]:
# build the vocablury of words
chars = sorted(list(set("".join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi["."] = 0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(itos)


In [15]:
# build the dataset

block_size = 3

def build_dataset(words):
    x,y = [], []

    for w in words:
        context = [0] * block_size
        for ch in w + ".":
            ix = stoi[ch]
            x.append(context)
            y.append(ix)
            context = context[1:] + [ix]

    x = torch.tensor(x)
    y = torch.tensor(y)

    return x, y


import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

xtr, ytr = build_dataset(words[:n1])
xdev, ydev = build_dataset(words[n1:n2])
xte, yte = build_dataset(words[n2:])




In [16]:
# utility function to be used later to compare the gradients we calculated

def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f"{s:15s} | exact : {str(ex):5s} | approximate : {str(app):5s} | maxdiff : {maxdiff}")




In [18]:
# init the layers
n_emb = 10
n_hidden = 64

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_emb), generator=g)

W1 = torch.randn((n_emb * block_size, n_hidden), generator = g) * (5/3) / ((n_emb * block_size)**0.5)
b1 = torch.randn((n_hidden), generator = g) * 0.1

W2 = torch.randn((n_hidden, vocab_size), generator = g) * 0.1
b2 = torch.randn((vocab_size), generator = g) * 0.1

# batch norm params
bngain = torch.ones((1, n_hidden)) * 0.1 + 1.9
bnbias = torch.zeros((1, n_hidden))

parameters = [C, W1, b1, W2, b2, bngain, bnbias]

print("total params : ", sum(p.nelement() for p in parameters))

for p in parameters:
    p.requires_grad = True

total params :  4137


In [25]:
batch_size = 32
n = batch_size

ix = torch.randint(0, xtr.shape[0], (batch_size, ), generator=g)
xb, yb = xtr[ix], ytr[ix]
print(ix.shape, xtr[ix].shape, ytr[ix].shape)

torch.Size([32]) torch.Size([32, 3]) torch.Size([32])


In [40]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.4048, grad_fn=<NegBackward0>)

In [169]:
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), yb] = -1.0/n

dprobs = (1/probs) * dlogprobs

dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)

dcounts = counts_sum_inv * dprobs
dcounts_sum = (-1 / counts_sum**2) * dcounts_sum_inv

dcounts += torch.ones_like(counts) * dcounts_sum

dnorm_logits = counts * dcounts

dlogit_maxes = (-1 * dnorm_logits).sum(1, keepdim=True)

dlogits = dnorm_logits.clone()
dlogits +=  F.one_hot(logits.max(1)[1], num_classes=logits.shape[1]) * dlogit_maxes

dh = dlogits @ W2.T

dW2 = h.T @ dlogits

db2 = dlogits.sum(0, keepdim=True)

dhpreact = (1 - h**2) * dh

dbngain = (bnraw * dhpreact).sum(0, keepdim=True)

dbnbias = dhpreact.sum(0, keepdim=True)

dbnraw = (bngain * dhpreact)

dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)

dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv 

dbndiff2 = torch.ones_like(bndiff2) * dbnvar * (1.0/(n-1))

dbndiff =  bnvar_inv * dbnraw
dbndiff += (2 * bndiff) * dbndiff2

dbnmeani = (-1.0 * dbndiff).sum(0)

dhprebn = dbndiff.clone()
dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani) 

dembcat = dhprebn @ W1.T

dW1 = embcat.T @ dhprebn

db1 = dhprebn.sum(0, keepdim = True)

demb = dembcat.view(emb.shape)

dC = torch.zeros_like(C)

for k in range(xb.shape[0]):
    for j in range(xb.shape[1]):
        ix = xb[k,j]
        dC[ix] += demb[k,j]


print(C.shape)
print(emb.shape)
print(bnmeani.shape)

cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

torch.Size([27, 10])
torch.Size([32, 3, 10])
torch.Size([1, 64])
logprobs        | exact : True  | approximate : True  | maxdiff : 0.0
probs           | exact : True  | approximate : True  | maxdiff : 0.0
counts_sum_inv  | exact : True  | approximate : True  | maxdiff : 0.0
counts_sum      | exact : True  | approximate : True  | maxdiff : 0.0
counts          | exact : True  | approximate : True  | maxdiff : 0.0
norm_logits     | exact : True  | approximate : True  | maxdiff : 0.0
logit_maxes     | exact : True  | approximate : True  | maxdiff : 0.0
logits          | exact : True  | approximate : True  | maxdiff : 0.0
h               | exact : True  | approximate : True  | maxdiff : 0.0
W2              | exact : True  | approximate : True  | maxdiff : 0.0
b2              | exact : True  | approximate : True  | maxdiff : 0.0
hpreact         | exact : True  | approximate : True  | maxdiff : 0.0
bngain          | exact : True  | approximate : True  | maxdiff : 0.0
bnbias          | exact :